In [18]:
import requests
import json
import pandas as pd

HOST = 'https://www.speedrun.com/api/v1/'

In [66]:
# get game id for majora's mask

games = requests.get('https://www.speedrun.com/api/v1/games?abbreviation=mm').text
game_dict = json.loads(games)['data'][0]

game_id = game_dict['id']
game_id

'268zey6p'

In [ ]:
# TEST BLOCK ADDED 1/25
game = requests.get(HOST + 'runs' + '?embed=players')
game.json()


In [51]:
# get all runs done for mm (max 200 at a time)

runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
runs_dict1 = json.loads(runs)
runs_list = json.loads(runs)['data']
runs_dict1['pagination']['size'] # max elements

# new way of getting player id from run 2 (or any run) WHILE getting run
runs_dict1['data'][2]['players']['data'][0]['names']['international']

# get name of player without an account
runs_dict1['data'][12]['players']['data'][0]['name']

'Nooya'

In [69]:
# Successfull functions for first version

def make_list():
  full_list = []
  offset = 0
  runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
  dict = json.loads(runs)

  while dict['pagination']['size'] == 200:
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id + '&offset=' + str(offset)).text
    dict = json.loads(runs)
    full_list.extend(dict['data'])
    offset += 200

  return full_list


def add_time():
  runs = make_list()
  player_times = {}

  for i in range(len(runs)):
    if runs[i]['players']['data'][0].get('names'):
      player = runs[i]['players']['data'][0]['names']['international']
    else:
      player = runs[i]['players']['data'][0]['name']


    time = runs[i]['times']['primary_t']

    if player in player_times:
      player_times[player] += time
    else:
      player_times[player] = time

  return player_times


In [87]:

def convert2(seconds):
  m, s = divmod(seconds, 60)
  h, m = divmod(m, 60)

  return str(int(h)) + ':' + str(int(m)) + ':' + str(int(s))

In [ ]:
final_data = add_time()
df = pd.DataFrame.from_dict({'Username':final_data.keys(), 'Total Time':final_data.values()})
df.sort_values(by=['Total Time'], ascending=False, inplace=True)
df['Total Time'] = df['Total Time'].apply(convert2)
df.reset_index(inplace=True, drop=True)
df

# Part 2: Generalizing to Grab Many Zelda Games

In [163]:
acronyms = ['mm', 'oot', 'tww', 'tp', 'alttp']
zelda = requests.get(HOST + 'games?name=zelda&max=100')
zelda_dict = zelda.json()['data']

# view all zelda games
zelda_games = []
for i in range (len(zelda_dict)):
    zelda_games.append(zelda_dict[i]['names']['international'])

In [181]:
GAME_SERIES = 'zelda'

def get_games(GAME_SERIES):
    id_map = {}
    request = requests.get(HOST + 'games?max=100&name=' + GAME_SERIES)
    zelda_list = request.json()['data']

    for i in range (len(zelda_list)):
        id_map[zelda_list[i]['id']] = zelda_list[i]['names']['international']

    return id_map

id_to_game = get_games(GAME_SERIES)

In [182]:
def new_make_list(game_id):
    full_list = []
    offset = 0
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
    dict = json.loads(runs)

    while dict['pagination']['size'] == 200:
        runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id + '&offset=' + str(offset)).text
        dict = json.loads(runs)
        full_list.extend(dict['data'])
        offset += 200

    return full_list

# new method, make new list entry for each person, time, and game
# at the very end, sum time grouped by person and game
def add_runs(game_id):
    game_id = '268zey6p'
    runs = new_make_list(game_id)
    usernames = []
    times = []
    games = []


    for i in range(len(runs)):
        if runs[i]['players']['data'][0].get('names'):
            usernames.append(runs[i]['players']['data'][0]['names']['international'])
        else:
            usernames.append(runs[i]['players']['data'][0]['name'])

        times.append(runs[i]['times']['primary_t'])
        games.append(id_to_game[game_id])

    player_times = {
        'username' : usernames,
        'time' : times,
        'game' : games,
    }

    return player_times

In [218]:
# steps for transforming dict into final df
df = pd.DataFrame(add_runs('stuff'), columns = ['username', 'time', 'game'])
df = df.groupby(['username', 'game'], as_index=False).sum()
df.sort_values(by=['time'], ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,username,game,time
0,TrevPerson,The Legend of Zelda: Majora's Mask,1270513.00
1,fullgrowngaming,The Legend of Zelda: Majora's Mask,976611.00
2,dopeZera,The Legend of Zelda: Majora's Mask,700351.00
3,popesquidward,The Legend of Zelda: Majora's Mask,553996.81
4,EnNopp112,The Legend of Zelda: Majora's Mask,551924.47
...,...,...,...
480,kemXtrm,The Legend of Zelda: Majora's Mask,1780.00
481,BetoKos,The Legend of Zelda: Majora's Mask,1595.00
482,Marthur,The Legend of Zelda: Majora's Mask,759.00
483,Perzyk,The Legend of Zelda: Majora's Mask,133.00
